# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating arguments with your requests. 

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.  

In this chat scenario, as the user talks back and forth with the bot, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. 

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.0.1

Let's define a prompt outlining a dialogue chat bot.

In [2]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

Register your semantic function

In [3]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Initialize your arguments

In [4]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

Chat with the Bot

In [5]:
var userInput = "Hi, I'm looking for book suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

Update the history with the output and set this as the new input value for the next request

In [6]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Sure! What genre or type of book are you interested in?



Keep Chatting!

In [7]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;
    
    // Show the response
    Console.WriteLine(result);
};

In [8]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");


User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: I would recommend "The Greeks: History, Culture, and Society" by Ian Morris and Barry B. Powell.



In [9]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");


User: that sounds interesting, what are some of the topics I will learn about?
AI: In "The Greeks: History, Culture, and Society," you will learn about the history of ancient Greece, including its political systems, wars, and famous figures such as Alexander the Great. You will also learn about Greek culture, including their art, literature, and philosophy. The book covers topics such as the development of democracy, the rise and fall of the Greek city-states, and the influence of Greek culture on the Western world.



In [10]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");


User: Which topic from the ones you listed do you think most people find interesting?
AI: I think many people find the topic of Greek philosophy to be particularly interesting. Greek philosophers such as Socrates, Plato, and Aristotle made significant contributions to the fields of ethics, logic, and metaphysics, and their ideas continue to influence Western thought today.



In [11]:
await Chat("could you list some more books I could read about the topic(s) you mentioned?");


User: could you list some more books I could read about the topic(s) you mentioned?
AI: Sure! Here are some more books you could read about Greek history and culture:

1. "The Peloponnesian War" by Thucydides
2. "The Iliad" by Homer
3. "The Odyssey" by Homer
4. "The Histories" by Herodotus
5. "The Rise and Fall of Athens: Nine Greek Lives" by Plutarch
6. "The Birth of Classical Europe: A History from Troy to Augustine" by Simon Price and Peter Thonemann
7. "The Greek World After Alexander 323-30 BC" by Graham Shipley
8. "The Greek Myths" by Robert Graves
9. "The Ancient Greeks: An Introduction" by Stephanie Lynn Budin
10. "The Oxford Illustrated History of Greece and the Hellenistic World" edited by John Boardman, Jasper Griffin, and Oswyn Murray

These books cover a range of topics within Greek history and culture, from specific events and individuals to broader overviews of the ancient Greek world.



After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [12]:
Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Sure! What genre or type of book are you interested in?

User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: I would recommend "The Greeks: History, Culture, and Society" by Ian Morris and Barry B. Powell.

User: that sounds interesting, what are some of the topics I will learn about?
AI: In "The Greeks: History, Culture, and Society," you will learn about the history of ancient Greece, including its political systems, wars, and famous figures such as Alexander the Great. You will also learn about Greek culture, including their art, literature, and philosophy. The book covers topics such as the development of democracy, the rise and fall of the Greek city-states, and the influence of Greek culture on the Western world.

User: Which topic from the ones you listed do you think most people find interesting?
AI: I think many people find the topic of Greek philosophy to be particularly interest